In [1]:
import pandas as pd
import numpy as np
from datetime import date
from functools import partial
from multiprocessing import cpu_count, Pool
from extract_msg import Message
# from pprint import pprint
import zipfile as zf
import glob
import re
import shutil
import os
import sys


In [2]:
def unzip(path_origen: str, path_dest: str):
    '''
    :param path_origen: Path de la carpeta d'origen del conjunt de
    dades original en format ZIP.
    :param path_dest: Path de la carpeta de destinació de les
    dades descomprimides.
    
    En aquest cas, aquesta funció no retornarà cap valor,
    doncs només executa una tasca concreta.
    '''  

    with zf.ZipFile(path_origen, 'r') as zip_f:
        # Descomprimim tot el contingut del zip
        zip_f.extractall(path_dest)


def read_msg(file):
    '''
    :param file: Path de cadascun dels arxius generats en la
    iteració executada per la funció desplegament.
    
    :return: Retorna un diccionari amb l'assumpte (Subject) i el cos
    (Body) corresponents a cada missatge en format MSG i que
    formaran part de la llista de diccionaris.
    '''  
    msg = Message(file)
    dicc = {'subj': msg.subject, 'body': msg.body}

    return dicc
 

def read_other_files(folder):
    '''
    :param folder: Path de la carpeta que conté els fitxers CSV
    amb dades complementàries.

    :return: Retorna un dataframe amb la relació de temes calssificats per
    barris i un diccionari amb la relació de codi de reunió amb
    la seva respectiva data de realització.
    '''  
    ID_filename = "ID_Dates.csv"
    path_ID_file = os.path.join(folder, ID_filename) 
    ID_Dates = pd.read_csv(path_ID_file)
    dicc_ID_Dates = ID_Dates.to_dict(orient='index')
    # Emprem el paràmetre "index" doncs conserva l'índex com claus del
    # diccionari de diccionaris resultant. V. doc. ofical:
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html
    # No val la pena convertir-ho en llista de diccionaris, tal i com es
    # podria fer amb una llista comprimida: https://stackoverflow.com/a/49248618
    barris_filename = "temes_barris.csv"
    path_temes_file = os.path.join(folder, barris_filename) 
    temes_barris = pd.read_csv(path_temes_file)


    return dicc_ID_Dates, temes_barris


def remove(path_origen: str, path_folders: list):
    '''
    :param path: Path de la carpeta que conté les carpetes a
    eliminar.

    En aquest cas, aquesta funció no retornarà cap valor,
    doncs només executa una tasca concreta.
    '''  
    
    for folder in path_folders:
        path_delete = path_origen + "/" + str(folder) + "/"
    # https://www.geeksforgeeks.org/python-os-path-isdir-method/
        if os.path.isdir(path_delete):
            shutil.rmtree(path_delete, ignore_errors=True)
            print("La carpeta %s s'ha eliminat amb èxit." % path_delete)
        else:
            print("Error: la carpeta %s no existeix." % path_delete)




In [3]:

def desplegament(carpetes: list):
    '''
    :param carpetes: Llista de noms de carpetes amb les que
                    s'executaran diverses tasques.
    
    :return: Aquesta funció retorna els següents valors:
            - 
            - 
            - 
            - 
    '''  
    zipfiles = os.path.join(carpetes[0], "*.zip")

    for fin in glob.glob(zipfiles):
        unzip(fin, carpetes[0])

    # Filtrem els continguts de la carpeta origen de manera
    # que no enumeri en la llista els fitxers que també hi
    # siguin presents. Font: https://stackoverflow.com/a/72039309 
    folders = [name for name in os.listdir(carpetes[0]) if os.walk(name)]
    zips = ".zip"
    folders = [f for f in folders if all(z not in f for z in zips)]


    # En primer lloc, volem comprovar si la màquina que executa el
    # programa disposa d'una o més CPU. Per fer aquest avaluació
    # mitjançant la funció cpu_count ens inspirem en aquest exemple
    # del site StackOverflow: https://stackoverflow.com/a/1006337
    # En el cas de només disposar d'1 CPU, llavors realitzarem una
    # execució seqüencial.
    if cpu_count() == 1:
        for f in folders:
            msgfiles = os.path.join(carpetes[0], f, "*.msg")        
            list_dicc = [read_msg(fin) for fin in glob.glob(msgfiles)]

            subcarpeta = os.path.join(carpetes[0], f)
            files = read_other_files(subcarpeta)
    # En el cas de disposar-ne de 2 o més, llavors realitzarem un
    # paral·lelisme amb els disponibles.
    elif cpu_count() >= 2:
        # Per implementar un multiprocess per llegir fitxers, adaptem
        # aquesta proposta del site StackOverflow:
        # https://stackoverflow.com/a/36590187
        # En el cas de disposar de 2 CPU, s'obté una millora a 2'9 segons
        # en contrast als 3'9 segons de la execució seqüencial.

        for f in folders:
            msgfiles = os.path.join(carpetes[0], f, "*.msg")
            files = [fin for fin in glob.glob(msgfiles)]

            with Pool(processes=cpu_count()) as pool:
                list_dicc = pool.map(read_msg, files)

            subcarpeta = os.path.join(carpetes[0], f)
            files = read_other_files(subcarpeta)

    set_subj = set([d['subj'] for d in list_dicc])

    class NumberMSG_Exception(Exception):
        pass

    try:
        # Creem una excepció per que aturi el program i esborri els fitxers generats
        # en el cas que es detecti que no coincideix el nombre l'Ordres del dia amb
        # el d'Actes diferents. Fonts:
        # https://stackoverflow.com/a/49953661
        # https://medium.com/@saadjamilakhtar/5-best-practices-for-python-exception-handling-5e54b876a20             
        if len(set_subj) % 2 != 0:
            print("Aquest error s'origina per que no coincideix el nombre de fitxers\n" \
                "MSG amb Ordres del dia amb els d'Actes de reunions. Comprova que:\n" \
                "1. Que el nombre total de fitxers sigui parell.\n" \
                "2. Si el nombre total de fitxers és parell, llavors que no hi hagi\n" \
                "algun fitxer duplicat per accident.")
            print(f"Detectem {len(list_dicc)} fitxers MSG " \
                f"i d'aquests {len(set_subj)} en són diferents.")
            raise NumberMSG_Exception("Error detectat en la carpeta de fitxers MSG.")      

    except NumberMSG_Exception:
        # Eliminem els continguts de la carpeta data:
        remove(carpetes[0], folders)
        # Finalitzem la execució del robot:
        sys.exit(1)
    
    else:
        # D'altra manera, continuem amb normalitat la execució del robot:
        reunions = files[0]
        barris = files[1]  
        
        print(f"Hem recopilat {len(list_dicc)} MSG i {len(files)} CSV en total.")

        # Eliminem els continguts de la carpeta data:
        remove(carpetes[0], folders)

        # I, llavors, creem la subcarpeta específica per les dades processades:
        full_path = os.path.join(carpetes[0], carpetes[1], "")
        os.makedirs(full_path, exist_ok=True)

        return list_dicc, reunions, barris, full_path



In [4]:
def parse_dates(subj: str):
    '''
    :param subj: Input del text corresponent a l'assumpte
    de cada email.
    
    :return: Retorna la data de la reunió.
    '''  
    rgx = 'Mobilitat ([0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9])'
    data_reunio = re.search(rgx, subj).group(1)

    return data_reunio


def parse_referencies(body: str):
    '''
    :param body: Input del text corresponent al cos principal
    de cada email.

    :return: Retorna una llista amb les referències de cada punt
    recollit en l?ordre del dia o l'Acta de cada reunió.
    '''  
    rgx = r'[0-9][0-9][0-9][0-9]/[0-9][0-9]/TTM|[0-9][0-9][0-9]/[0-9][0-9]/TTM'
    referencies = re.findall(rgx, body)

    return referencies


def parse_continguts (body: str):
    '''
    :param body: Input del text corresponent al cos principal
    de cada email.

    :return: Retorna una llista amb el text corresponent a la descripció
    del tema (una oració) el detall del tema en qüestió.
    '''  
    rgx = r'[0-9][0-9][0-9][0-9]/[0-9][0-9]/|[0-9][0-9][0-9]/[0-9][0-9]/'
    particions = re.split(rgx, body)
    
    # Eliminem l'encapçalament corresponent a cada reunió doncs no conté 
    # informació rellevant per la nostra tasca:
    n_particions = len(particions)
    particions_depurades = particions[1:n_particions]

    return particions_depurades

def split_continguts(contingut: list, patro: str):
    '''
    :param body: Input del text corresponent al cos principal
    de cada email.
    :param patro: Patró que delimita el fragment de text corresponent a
    la descripció del tema (una oració) amb el fragment corresponent amb el
    detall del tema en qüestió.

    :return: Retorna dues llistes (a, b): 
    - Llista a: Amb el fragment de text corresponent a la descripció del tema
    (una oració)
    - Llista b: Amb el fragment corresponent amb el detall del tema en qüestió.
    '''
    llista = [ele.split(patro, 1) for ele in contingut]

    index = range(0, len(llista))

    a = [llista[i][0] for i in index]
    b = [llista[i][1] for i in index if len(llista[i]) == 2]

    return a, b
    

def parsing_ttm(msg: dict, col_name: str, patro: str):
    '''
    :param dict: Diccionari que conté l'Assumpte i el cos principal
    de text de cada email.
    :param col_name: Input de la etqiueta de columna corresponent
    a si es tracta de la "Descripció de l'Ordre del dia" o bé "Acords".
    :param patro: Patró que delimita el fragment de text corresponent a
    la descripció del tema (una oració) amb el fragment corresponent amb el
    detall del tema en qüestió.
    :return: Retorna una llista amb el text corresponent a la descripció
    del tema (una oració) el detall del tema en qüestió.
    '''   
    msg['Data de la reunió'] = parse_dates(msg['subj'])
    msg['Id'] = parse_referencies(msg['body'])
    msg['Contingut'] = parse_continguts(msg['body'])
    
    # Atès que es detecta una errada tipogràfica prou freqüent,
    # resulta millor substituir-la de forma automatitzada.
    msg['Contingut'] = [ele.replace(" \r\n\r\n", patro + " ") for ele in msg['Contingut']]
    msg['Descripció del tema'], msg[col_name] = split_continguts(msg['Contingut'],
                                                                 patro)
    
    return msg


In [5]:
# Funció a la que es fa la crida des del script main.py:
def parsing(list_msg: list, OdD_str: str, acta_str: str):
    '''
    :param list_msg: Llista de diccionaris que conté cadascun
    l'Assumpte i el cos principal de text de cada email.
    :param OdD_str: Literal necessari per construir el patró
    regex per diferenciar entre missatges que són Ordres del dia
    o Actes de reunions.
    :param acta_str: Literal necessari per construir el patró
    regex per diferenciar entre missatges que són Ordres del dia
    o Actes de reunions.
    :return: Retorn dues llistes de diccionaris d'emails d'Ordres
    del dia i Actes de reunions, respectivament.
    '''
    rgx_OdD = "^" + OdD_str
    rgx_Acta = "^" + acta_str
    patro = ["Descripció:", "Acords:"]

    col_name = ["Descripció Ordre del dia", "Acords"]

    ODs = [parsing_ttm(m, 
                       col_name[0],
                       patro[0]) for m in list_msg if re.search(rgx_OdD,
                                                         m['subj'])]
    
    Actes = [parsing_ttm(m,
                         col_name[1],
                         patro[1]) for m in list_msg if re.search(rgx_Acta,
                                                           m['subj'])]

    return ODs, Actes


In [6]:
def replace_continguts(dicc: dict, cols_names: list, literals: list):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.
    :param literals: Llista de patrons literals a eliminar.
    
    :return: Diccionari amb els patrons ja suprimits.
    '''
    
    for col in cols_names:
        
        for p_literal in literals:
            dicc[col] = [s.replace(p_literal, '') for s in dicc[col]]

    return dicc


def sub_continguts(dicc: dict, cols_names: list, regex: list):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.
    :param regex: Llista de patrons regex a eliminar.

    :return: Diccionari amb els patrons ja suprimits.
    '''    
    for col in cols_names:
    
        for p_regex in regex:
            dicc[col] = [re.sub(p_regex, '', s) for s in dicc[col]]

    return dicc

def replace_typos(dicc: dict, cols_names: list, literals: dict):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.
    :param literals: Diccionari de deccionaris els patrons literals a substituir
                (key) i el respectiu string que l'ha de substiuir (values).
    
    :return: Diccionari amb els patrons ja suprimits.
    '''
    
    for col in cols_names:
        
        for p_literal, string in literals.items():
            dicc[col] = [s.replace(p_literal, string) for s in dicc[col]]

    return dicc


def sub_typos(dicc: dict, cols_names: list, regex: dict):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.
    :param literals: Diccionari de deccionaris els patrons regex a substituir
                (key) i el respectiu string que l'ha de substiuir (values).
    
    :return: Diccionari amb els patrons ja suprimits.
    '''    
    for col in cols_names:
    
        for p_regex, string in regex.items():
            dicc[col] = [re.sub(p_regex, string, s) for s in dicc[col]]

    return dicc


def separar_paraules(dicc: dict, cols_names:list):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.

    :return: Diccionari amb les potencials paraules (minúsculaMajúscula).
    '''

    for col in cols_names:
        # Comprehension list que executa la separació de paraules segons el
        # patró ("minúsculaMajúscula"). Font (optem pel mètode #2):
        # https://www.geekforgeeks.org/python-add-space-between-potential-words/
        # i per l'ús de lookarounds i avaluacions per gestionar excepcions
        # consultem aquesta solució proposada en StackOverflow:
        # https://stackoverflow.com/a/25674758
        dicc[col] = [re.sub(r"(?<=[a-z])(?=[A-Z])", r" ", s) for s in dicc[col]]
        # Generem dos grups separats per un espai en blanc pels casos d'una minúscula
        # que s'ubiqui a continuació d'una paraula en majúscules i acabi en majúscula,
        # com pot ser una siga o un acrònim (patró "MAJÚSCULESminúscules"):
        dicc[col] = [re.sub(r"([A-Z]+[A-Z$])([a-z])", r"\1 \2", s) for s in dicc[col]]
        # Per afegir un espai en blanc entre un punt de puntuació i una paraula:
        # https://stackoverflow.com/a/44263500
        dicc[col] = [re.sub(r"(?<=[.,:;?!])(?=[^\s])", r" ", s) for s in dicc[col]]
    
    return dicc


def cleaning(dicc: dict, col_name: str, patrons: list):
    '''
    :param dicc: Diccionari amb els diversos camps dels emails ja parsejats.
    :param cols_names: Llista amb les etiquetes de columna de les variables
    on s'ha d'han executar les tasques de neteja del text.
    :param patrons: Llista amb cinc llistes patrons literals i regex a
    eliminar; actualment només es fa ús de quatre.

    :return: Diccionari amb els patrons ja suprimits.
    '''

    # Normalització i eliminació d'epígrafs innecessaris:
    cols_names = ["Descripció del tema", col_name]
    dicc = replace_continguts(dicc, cols_names, patrons[0])
    dicc = sub_continguts(dicc, cols_names, patrons[1])
    # Correció d'errors ortotipogràfics
    dicc = replace_typos(dicc, cols_names, patrons[2])
    dicc = sub_typos(dicc, cols_names, patrons[3])
    # Separació de potencials paraules amb altres paraules i els signes
    # de puntuació:
    dicc = separar_paraules(dicc, cols_names)

    return dicc


In [7]:
pttrns_typos_literals = {'economica': 'econòmica', 'TTM': 'Taula tècnica',
                         "taula tècnica": "Taula tècnica", "taula Tècnica": "Taula tècnica",
                         "Taula Tècnica": "Taula tècnica",
                "Guardia urbana": "Guàrdia Urbana", "Guardia Urbana": "Guàrdia Urbana",
                "GUB": "Guàrdia Urbana", "àrea verda": "Àrea Verda",
                "Àrea verda": "Àrea Verda", "àrea Verda": "Àrea Verda",
                "la escola": "l'escola", "Gran vista": "Gran Vista", 
                "brigaa": "Brigada", "brigada": "Brigada", "Psg.": "Pg.",
                "gerència": "Gerència",  "acordos": "acords", "vehícle": "vehicle",
                "respotà": "resposta", "enviaà": "enviarà",
                "St. Alejandro": "Sant Alexandre",
                "Olimpic": "Olímpic", "arenys": "Arenys", "Roquetas": "Roquetes",
                "Carmen C.": "Carmen Castaño",
                "Cristina G.": "Cristina Gil", "Raúl O.": "Raúl Ortega", 
                "Montse V.": "Montse Vico", "Jaume S.": "Jaume Sauch",
                "SSTT Dte.": "Serveis tècnics del Districte", "SSTT": "Serveis Tècnics"}

for k, v in pttrns_typos_literals.items():
    print(f"- {k}: {v}")
    

- economica: econòmica
- TTM: Taula tècnica
- taula tècnica: Taula tècnica
- taula Tècnica: Taula tècnica
- Taula Tècnica: Taula tècnica
- Guardia urbana: Guàrdia Urbana
- Guardia Urbana: Guàrdia Urbana
- GUB: Guàrdia Urbana
- àrea verda: Àrea Verda
- Àrea verda: Àrea Verda
- àrea Verda: Àrea Verda
- la escola: l'escola
- Gran vista: Gran Vista
- brigaa: Brigada
- brigada: Brigada
- Psg.: Pg.
- gerència: Gerència
- acordos: acords
- vehícle: vehicle
- respotà: resposta
- enviaà: enviarà
- St. Alejandro: Sant Alexandre
- Olimpic: Olímpic
- arenys: Arenys
- Roquetas: Roquetes
- Carmen C.: Carmen Castaño
- Cristina G.: Cristina Gil
- Raúl O.: Raúl Ortega
- Montse V.: Montse Vico
- Jaume S.: Jaume Sauch
- SSTT Dte.: Serveis tècnics del Districte
- SSTT: Serveis Tècnics


In [8]:
# Bloc #1 d'execució (main):
carpetes = ['data', 'data_processat', 'reports']
strings = ['Ordre del dia', 'Acta']
cols_names = ["Descripció Ordre del dia", "Acords"]
patrons_literals = ['\n', '\r', '\t', 'PROTOCOLS DE GUÀRDIA URBANA',
                    "SUPERILLA D'HORTA", "CAMPANYA DE MOTOS EN VORERA",
                    "CAMPANYES DE MOTOS EN VORERA", "PUNTS INFORMATIUS",
                    'Descripció:', "Acords:",
                    ]
patrons_regex = ["^TTM ", "^TTM:", "^- ", "^  ", "^ ", '  $', ' $', "^[0-9][0-9]/2[0-9]",
                 r'MOBILITAT[^>]+PUJOLET\)']
# patrons_regex = ["^TTM ", "^TTM: ", "^- ", "^[0-9][0-9]/2[0-9]"]
pttrns_typos_literals = {'economica': 'econòmica', 'TTM': 'Taula tècnica',
                         "taula tècnica": "Taula tècnica", "taula Tècnica": "Taula tècnica",
                         "Taula Tècnica": "Taula tècnica",
                "Guardia urbana": "Guàrdia Urbana", "Guardia Urbana": "Guàrdia Urbana",
                "GUB": "Guàrdia Urbana", "àrea verda": "Àrea Verda",
                "Àrea verda": "Àrea Verda", "àrea Verda": "Àrea Verda",
                "la escola": "l'escola", "Gran vista": "Gran Vista", 
                "brigaa": "Brigada", "brigada": "Brigada", "Psg.": "Pg.",
                "gerència": "Gerència",  "acordos": "acords", "vehícle": "vehicle",
                "respotà": "resposta", "enviaà": "enviarà",
                "St. Alejandro": "Sant Alexandre",
                "Olimpic": "Olímpic", "arenys": "Arenys", "Roquetas": "Roquetes",
                "Carmen C.": "Carmen Castaño",
                "Cristina G.": "Cristina Gil", "Raúl O.": "Raúl Ortega", 
                "Montse V.": "Montse Vico", "Jaume S.": "Jaume Sauch",
                "SSTT Dte.": "Serveis tècnics del Districte", "SSTT": "Serveis Tècnics"}
pttrns_typos_regex = {r"(\bdistricte\b)": r"Districte", r"(\bmols\b)": r"molts",
                      r"(\bambar\b)": r"àmbar", r"(\bC\b/)": r"c.",
                      r"(\bMF\b)": r"Manuel Franco", r"(\bde Gràcies\b)": r"de Gràcia",
                      r"\btaule\b": r"Taula", r"\bcarrertera\b": r"carretera"
                      }

# De moment, no està activa aquesta variable:
pttrns_null = ["tancar el tema", "tancat el tema", "Tancar tema", "tancar ambdós temes",
               "es dóna per tancat", "es dona per tancat", "donar per tancat",
               "ja estava executat", "tema que ha treballat",
               "Mateix que en el tema", "Pendent de visita", "Per valorar en la propera",
               "Protocol fet i ", "Protocol fet pendent", "Pendent d'execució",
               "pendent d'execució", "pendent de rebre", "Pendent de pressupost"
               "Pendent de modificar", "està treballant en la", "Pendent de fer el Protocol",
               "treballant en el Protocol", "protocol està fet", "ja s'ha executat",
               "ja està executat", "encarregarà el Protocol",
               "La Taula aprova el", "S'aprova per la Taula", "tramitar i executar el Protocol",
               "s'aprova el Protocol", "s'aprova la proposta de nou Protocol",
               "quan hi hagin novetats", "Pendent de conèixer", "Resta pendent", "resta pendent"
               "es gestionarà per correu electrònic",
               "Sense novetats", "No hi ha novetat",
               "Es tractarà el tema a la propera",
               "No va donar temps", "Aquesta tema no estava inclòs", "No dóna temps per",
               "No va haver prou temps", "No dóna prou temps",
               "Es valora com tema tractat",
               "La Taula es dóna per informada"]
patrons = [patrons_literals, patrons_regex, pttrns_typos_literals, pttrns_typos_regex,
           pttrns_null]

list_dicc, reunions, barris, path_proc = desplegament(carpetes[0:2])

# "reunions" (diccionari de diccionaris) i "barris" (DataFrame) actuaran
# com variables globals pel següent bloc.

ODs, Actes = parsing(list_dicc, strings[0], strings[1])

print(f"S'han compilat les dades de {len(ODs)} ordres del dia de reunions.")
print(f"S'han compilat les dades de {len(Actes)} actes de reunions.")

ODs = [cleaning(d, cols_names[0], patrons) for d in ODs]
Actes = [cleaning(d, cols_names[1], patrons) for d in Actes]


Hem recopilat 100 MSG i 2 CSV en total.
La carpeta data/TTM/ s'ha eliminat amb èxit.
S'han compilat les dades de 50 ordres del dia de reunions.
S'han compilat les dades de 50 actes de reunions.


In [9]:
print(path_proc)

data/data_processat/


In [10]:
test_list = ["babauMarc", "MobilitatTaula", "LaCasadeMarc", "AgentGUB",
             "failBIMSA", "PASSAT DE VOLTESno, lo següentADÉU!",
             "hola,cagada!Pastoret"]



res = [re.sub(r"(?<=[a-z])(?=[A-Z])", r" ", ele) for ele in test_list]


res = [re.sub(r"([A-Z]+[A-Z$])([a-z])", r"\1 \2", ele) for ele in res]


res = [re.sub(r"(?<=[.,:;?!])(?=[^\s])", r" ", ele) for ele in res]

res

['babau Marc',
 'Mobilitat Taula',
 'La Casade Marc',
 'Agent GUB',
 'fail BIMSA',
 'PASSAT DE VOLTES no, lo següent ADÉU!',
 'hola, cagada! Pastoret']

In [11]:
string_ = ["Districte", "de districte", "districtes", " MF", "diu MF", "de Gràcies",
           " C/Floresta",
           "DC/ Floresta",
           "MOBILITAT FONT D'EN FARGUES (ZONA DELS CARRERS CANONGE ALMERA I CAN PUJOLET)"]

string_ = [re.sub(r"(districte\b)", r"Districte", ele) for ele in string_]
string_ = [re.sub(r"(\bMF\b)", r"Manuel", ele) for ele in string_]
string_ = [re.sub(r"(\bde Gràcies\b)", r"de Gràcia", ele) for ele in string_]
string_ = [re.sub(r"(\bC\b/)", r"C.", ele) for ele in string_]
string_ = [re.sub(r'MOB[^>]+PUJOLET\)', '', ele) for ele in string_]

string_


['Districte',
 'de Districte',
 'districtes',
 ' Manuel',
 'diu Manuel',
 'de Gràcia',
 ' C.Floresta',
 'DC/ Floresta',
 '']

In [12]:
# Emparellaments. Font:
# https://medium.com/@mr.stucknet/get-creative-with_nested-comprehensions-in-python_and_build-incredible-applications-53736701d6a4

pairs = [(string_[a], string_[b])
         for a in range(len(string_)) 
         for b in range(a, len(string_)) 
         if string_[a] != '' and string_[b] != '']

pairs

[('Districte', 'Districte'),
 ('Districte', 'de Districte'),
 ('Districte', 'districtes'),
 ('Districte', ' Manuel'),
 ('Districte', 'diu Manuel'),
 ('Districte', 'de Gràcia'),
 ('Districte', ' C.Floresta'),
 ('Districte', 'DC/ Floresta'),
 ('de Districte', 'de Districte'),
 ('de Districte', 'districtes'),
 ('de Districte', ' Manuel'),
 ('de Districte', 'diu Manuel'),
 ('de Districte', 'de Gràcia'),
 ('de Districte', ' C.Floresta'),
 ('de Districte', 'DC/ Floresta'),
 ('districtes', 'districtes'),
 ('districtes', ' Manuel'),
 ('districtes', 'diu Manuel'),
 ('districtes', 'de Gràcia'),
 ('districtes', ' C.Floresta'),
 ('districtes', 'DC/ Floresta'),
 (' Manuel', ' Manuel'),
 (' Manuel', 'diu Manuel'),
 (' Manuel', 'de Gràcia'),
 (' Manuel', ' C.Floresta'),
 (' Manuel', 'DC/ Floresta'),
 ('diu Manuel', 'diu Manuel'),
 ('diu Manuel', 'de Gràcia'),
 ('diu Manuel', ' C.Floresta'),
 ('diu Manuel', 'DC/ Floresta'),
 ('de Gràcia', 'de Gràcia'),
 ('de Gràcia', ' C.Floresta'),
 ('de Gràcia', 'DC

In [13]:
# %load_ext line_profiler

# Amb molta diferència, el gruix del temps d'execució correspon
# a la funció desplegament() i, concretament, a la execució de la
# subfunció read_msg().

# %lprun -f desplegament desplegament(carpetes[0])

# %lprun -f parsing parsing(list_dicc, strings[0], strings[1])

# %lprun -f cleaning [cleaning(d, cols_names[1], patrons) for d in Actes]

In [14]:
# %lprun -f cleaning [cleaning(d, cols_names[0], patrons) for d in ODs]

In [15]:
# pprint(ODs[2]) # Planteajr-me d'emprar aquesta funció:
# https://levelup.gitconnected.com/stop-using-print-every-time-in-python-use-pprint-instead-e8761b25048f
# També fer ús de format: print(f"oioio: {something}")

In [16]:
# Fragment de codi per debugging:
# for i in range(0, len(ODs)):
#    if len(ODs[i]['Descripció del tema']) == len(ODs[i]['Descripció Ordre del dia']):
#
#        print("Índex:", i, "OK", '\n')
#    else:
#        print("Índex:", i, ODs[i]['Data de la reunió'])
#        print('Descripció Ordre del dia:', len(ODs[i]['Descripció Ordre del dia']))
#        print('Descripció del tema:', len(ODs[i]['Descripció del tema']), '\n')


In [17]:
# for i in range(0, len(Actes)):
#    if len(Actes[i]['Descripció del tema']) == len(Actes[i]['Acords']):
#
#        print("Índex:", i, "OK", '\n')
#    else:
#        print("Índex:", i, Actes[i]['Data de la reunió'])
#        print('Acords:', len(Actes[i]['Acords']))
#        print('Descripció del tema:', len(Actes[i]['Descripció del tema']), '\n')


In [18]:
def to_df(diccionaris: list):
    '''
    :param diccionaris: Llista de diccionaris amb els diversos camps dels
    emails ja parsejats.

    :return: DataFrame.
    '''
    
    dfs = [pd.DataFrame(d) for d in diccionaris]
    df = pd.concat(dfs)

    return df


def gen_ref_reunions(df, reunions_lst: list):
    '''
    :param df: DataFrame d'input.
    :param reunions_lst: Llista de diccionaris amb la ID de cada
    reunió i la seva data corresponent:

    :return: DataFrame.
    '''

    # Generem la nova columna "Ref. Tema i Reunió":
    df['Ref. Tema i Reunió'] = df['Id'] + ' ' + df['Data de la reunió']
    # Realitzem la substitució:
    for d in reunions_lst:        
        df['Ref. Tema i Reunió'] = df['Ref. Tema i Reunió'].str.replace(d['Data'], d['ID_reunió'], regex=False)
    
    return (df)


def cleaning_df(df, buits:str, col: str):
    '''
    :param df: DataFrame d'input.
    :param buits: Frase tipus per substituir valors nuls i buits ('').
    :patam col: Etiqueta de columna.

    :return: DataFrame d'output.
    '''

    mask_buit = df[col]==''
    df[col][mask_buit] = buits
    df[col] = df[col].fillna(buits)

    return df



def manipulacio(ODs: list, Actes: list, buits: str, col: str):
    '''
    :param ODs: Llista de diccionaris amb les dades dels Ordres del dia
    de reunions.
    :param Actes: Llista de diccionaris amb les dades dels Ordres del dia
    de reunions.
    Variables globals: S'empren també com variables globals el diccionari
    amb diccionaris 

    :return: DataFrame ja llest per convertir-lo en el format ja desitjat.
    '''
    
    df_ODs = to_df(ODs)
    df_Actes = to_df(Actes)

    # Variable global reunions convertida a llista de diccionaris:
    reunions_lst = [d for d in reunions.values()]
    
    # Definim els dos DataFrame en una llista per facilitar la iteració
    # de la funció que genera la nova columna "Ref. Tema i Reunió":
    df_lst = [df_ODs, df_Actes] # Claus [0] i [1], respectivament.

    for df_i in df_lst:
        df_i = gen_ref_reunions(df_i, reunions_lst)

    # Reduïm el nombre de columnes de cada DataFrame prèviament al primer merge:
    df_lst[0] = df_ODs[['Ref. Tema i Reunió', 'Descripció Ordre del dia']]
    df_lst[1] = df_Actes[['Id', 'Ref. Tema i Reunió', 'Data de la reunió', 'Descripció del tema', 'Acords']]
    df_ = pd.merge(df_lst[0], df_lst[1],  how='right', left_on=['Ref. Tema i Reunió'], right_on=['Ref. Tema i Reunió'])
    
    # Executem el merge amb el DataFrame dels temes classificats per barris i
    # eliminem la columna "Id" que ara ja és redundant:
    actes = pd.merge(barris, df_,  how='right', left_on=['Id'], right_on=['Id'])
    actes = actes[['Ref. Tema i Reunió', 'Data de la reunió', 'Barri', 'Descripció del tema',
                   "Descripció Ordre del dia", 'Acords']]

    # Definim una màscara per identificar els espais en blanc ('') i substituim aquests 
    # i els valors nuls en la columna "Descripció Ordre del dia" per una frase tipus:

    actes = cleaning_df(actes, buits, col)

    # Convertim la columna "Data de la reunió" a format DateTime de pandas:
    actes['Data de la reunió'] = pd.to_datetime(actes['Data de la reunió'], dayfirst= True)

    # Per l'ús de la funció es pot consultar aquesta excel·lent exposició en
    # StackOverflow: https://stackoverflow.com/a/17141755 com també la doc.
    # oficial: 
    actes = actes.sort_values(by=['Data de la reunió', 'Ref. Tema i Reunió'], 
                              ascending=[True, True])

    print(f"S'han compilat les dades de {len(set(actes['Data de la reunió']))} reunions.")

    return actes

In [19]:
def output(df, file_name: str, extensio: list, path_dest: list):
    '''
    :param df: DataFrame input.
    :file_name: Plantilla del nom del fitxer. 

    En aquest cas, aquesta funció no retornarà cap valor,
    doncs només executa una tasca concreta.
    '''    
    # Definim la data d'avui...
    avui = date.today()
    # Per la manipulació del string de la data:
    # https://codefather.tech/blog/python-datetime-days/
    avui = avui.strftime("%Y%m%d")

    # ...i també hi inclourem la referència corresponent de la darrera
    # reunió a partir de la variable global "reunions":

    r = [d['ID_reunió'] for d in reunions.values()]
    ref = str(r[-1]).replace('/','-')

    nom_excel = str(avui) + ' ' + file_name + ' ' + ref + extensio[0]
    path_excel = path_dest[0] + "/" + nom_excel

    df.to_excel(path_excel, sheet_name = 'Actes TTM', index = False)
    print(f"S'ha creat el fitxer {nom_excel} \nen la carpeta {path_dest[0]}.")

    # I generem una versió també en format
    nom_pickle = str(avui) + ' ' + file_name + ' ' + ref + extensio[1]
    path_pickle = path_dest[1] + nom_pickle
    df.to_pickle(path_pickle)
    print(f"S'ha creat el fitxer {nom_pickle} \nen la carpeta {path_dest[1]}.")


In [20]:

str_buits = "Aquest tema no estava inclòs en l'Ordre del dia o no comptava amb cap descripció"
file_name = "Recopilació de les actes de la TTM fins la reunió"
extensio = [".xlsx", ".pkl"]

# El primer objecte prové de la llista de noms de carpetes del Bloc #º mentre
# que el segon objecte és el un valor que retorna de la funció desplegament():
carpetes_dest = [carpetes[2], path_proc]

actes_ = manipulacio(ODs, Actes, str_buits, cols_names[0])

# Fem ús de la funció partial a fi d'alleugerir un xic la entrada
# d'arguments a la funció output() segons es recomana en aquest
# article en LevelUpCoding (18/07/2023):
# https://levelup.gitconnected.com/the-ultimate-guide-to-writing-functions-in-python-12-best-practices-122a797883a6
output_p = partial(output, path_dest=carpetes_dest)
output_p(actes_, file_name, extensio)


S'han compilat les dades de 50 reunions.
S'ha creat el fitxer 20230908 Recopilació de les actes de la TTM fins la reunió 15-23.xlsx 
en la carpeta reports.
S'ha creat el fitxer 20230908 Recopilació de les actes de la TTM fins la reunió 15-23.pkl 
en la carpeta data/data_processat/.


In [21]:

# https://stackoverflow.com/a/51325861


In [22]:
def replacing(dicc: dict):
    dicc['body'] = dicc['body'].replace('\r', "")
    dicc['body'] = dicc['body'].replace('\n', "")

    return dicc

data = [replacing(dicc) for dicc in data]

df = pd.DataFrame.from_dict(data)
df.to_excel('reports/excel.xlsx')


NameError: name 'data' is not defined

In [ ]:
class Circle:
    pi = 3.14159
    
    def __init__(self, radius):
        self.radius = radius
    
    def calculate_area(self):
        return self.pi * self.radius ** 2
    
    @classmethod
    def modify_pi(cls, new_pi):
        cls.pi = new_pi

circle1 = Circle(5)
print(circle1.calculate_area())

Circle.modify_pi(3.14)
circle2 = Circle(7)
print(circle2.calculate_area())